# Влияние близости школ на пешеходный трафик и средний чек (Индекс_РИА_2024 > 70)

Этот ноутбук:
- фильтрует данные по условиям (Индекс_РИА_2024 > 70, is_season=1, traffic_flag=1, пеший трафик>0),
- формирует признаки has_school и size_group,
- считает отношения метрик у школ / не у школ,
- выполняет t-тесты Стьюдента (one-sample по отношениям и Welch по сырым данным).


In [19]:
import pandas as pd
from pathlib import Path

input_path = Path("Х5_with_region_index_2024_population_patched_with_flags.xlsx")

df = pd.read_excel(input_path)
print("Размер исходного датафрейма:", df.shape)
df.head()

Размер исходного датафрейма: (256711, 21)


,new_id,Месяц,Трафик,Средний чек,"Дата открытия, категориальный","Торговая площадь, категориальный",Населенный пункт,Регион,Численность населения,Количество домохозяйств,...,"Трафик авто, в час","Маркетплейсы, доставки, постаматы (100 м)",Медицинские уч. и аптеки (300 м),Школы (300 м),Остановки (300 м),Продуктовые магазины (500 м),Пятерочки (500 м),Индекс_РИА_2024,traffic_flag,is_season
0,0,10,59662,976.170936,Средний по возрасту,Средний,Абинск г,Краснодарский край,38231,728,...,146.4,0,0,0,0,0,0,76.58,1,0
1,0,5,56674,1025.462154,Средний по возрасту,Средний,Абинск г,Краснодарский край,38231,728,...,146.4,0,0,0,0,0,0,76.58,1,0
2,0,1,51488,1158.150890,Средний по возрасту,Средний,Абинск г,Краснодарский край,38231,728,...,146.4,0,0,0,0,0,0,76.58,1,0
3,3594,7,68039,1119.028697,Средний по возрасту,Средний,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,3827,768,...,406.0,5,1,0,0,1,0,61.98,1,1
4,3594,6,64878,1112.584778,Средний по возрасту,Средний,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,3827,768,...,406.0,5,1,0,0,1,0,61.98,1,1


## Шаг 1. Фильтрация по Индекс_РИА_2024 > 70, сезонности и качеству трафика

In [22]:
col_index = "Индекс_РИА_2024"
col_is_season = "is_season"
col_traffic_flag = "traffic_flag"
col_pedestrian = "Трафик пеший, в час"

df[col_index] = pd.to_numeric(df[col_index], errors="coerce")

mask_index = (df[col_index] > 30) & (df[col_index] < 70)
mask_season = df[col_is_season] == 1
mask_traffic_flag = df[col_traffic_flag] == 1
mask_pedestrian = df[col_pedestrian] > 0

base_df = df[mask_index & mask_season & mask_traffic_flag & mask_pedestrian].copy()
print("Размер после фильтрации:", base_df.shape)
base_df.head()

Размер после фильтрации: (36896, 21)


,new_id,Месяц,Трафик,Средний чек,"Дата открытия, категориальный","Торговая площадь, категориальный",Населенный пункт,Регион,Численность населения,Количество домохозяйств,...,"Трафик авто, в час","Маркетплейсы, доставки, постаматы (100 м)",Медицинские уч. и аптеки (300 м),Школы (300 м),Остановки (300 м),Продуктовые магазины (500 м),Пятерочки (500 м),Индекс_РИА_2024,traffic_flag,is_season
3,3594,7,68039,1119.028697,Средний по возрасту,Средний,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,3827,768,...,406.0,5,1,0,0,1,0,61.98,1,1
4,3594,6,64878,1112.584778,Средний по возрасту,Средний,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,3827,768,...,406.0,5,1,0,0,1,0,61.98,1,1
10,3594,8,74160,1275.068118,Средний по возрасту,Средний,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,3827,768,...,406.0,5,1,0,0,1,0,61.98,1,1
13,8781,7,40386,683.123700,Средний по возрасту,Средний,1-я Моква д,Курская обл,500,201,...,483.2,0,0,1,0,1,1,50.02,1,1
15,8781,8,39323,687.641711,Средний по возрасту,Средний,1-я Моква д,Курская обл,500,201,...,483.2,0,0,1,0,1,1,50.02,1,1


## Шаг 2. has_school и size_group

In [23]:
col_schools = "Школы (300 м)"
col_size_cat = "Торговая площадь, категориальный"

base_df[col_schools] = pd.to_numeric(base_df[col_schools], errors="coerce").fillna(0)
base_df["has_school"] = base_df[col_schools] > 0

size_map = {
    "Маленький": "small",
    "Средний": "medium",
    "Большой": "large_plus",
    "Очень большой": "large_plus",
}
base_df["size_group"] = base_df[col_size_cat].map(size_map)
base_df = base_df[base_df["size_group"].notna()].copy()
print("Размер после size_group:", base_df.shape)
base_df[["Населенный пункт", "Регион", col_size_cat, "size_group", col_schools, "has_school"]].head()

Размер после size_group: (36896, 23)


,Населенный пункт,Регион,"Торговая площадь, категориальный",size_group,Школы (300 м),has_school
3,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,Средний,medium,0,False
4,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,Средний,medium,0,False
10,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,Средний,medium,0,False
13,1-я Моква д,Курская обл,Средний,medium,1,True
15,1-я Моква д,Курская обл,Средний,medium,1,True


## Шаг 3. Агрегация по (НП, регион, size_group, has_school)

In [24]:
col_np = "Населенный пункт"
col_region = "Регион"
col_traffic = col_pedestrian
col_check = "Средний чек"

base_df[col_traffic] = pd.to_numeric(base_df[col_traffic], errors="coerce")
base_df[col_check] = pd.to_numeric(base_df[col_check], errors="coerce")

group_cols = [col_np, col_region, "size_group", "has_school"]

agg_df = (
    base_df
    .groupby(group_cols)
    .agg(
        mean_traffic=(col_traffic, "mean"),
        mean_check=(col_check, "mean"),
        count=(col_traffic, "size"),
    )
    .reset_index()
)
print("Размер agg_df:", agg_df.shape)
agg_df.head()

Размер agg_df: (3777, 7)


,Населенный пункт,Регион,size_group,has_school,mean_traffic,mean_check,count
0,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,medium,False,89.857143,1168.893864,3
1,1-я Моква д,Курская обл,medium,False,86.555556,1344.300361,3
2,1-я Моква д,Курская обл,medium,True,95.666667,670.241819,3
3,Абадзехская ст-ца,Адыгея Респ,small,False,71.666667,833.505861,3
4,Абаза г,Хакасия Респ,medium,False,321.846154,844.179205,3


## Шаг 4. Отношения (у школы / не у школы) по НП и размеру

In [25]:
records = []

for (np_name, region_name, size_group), sub in agg_df.groupby([col_np, col_region, "size_group"]):
    with_school = sub[sub["has_school"] == True]
    without_school = sub[sub["has_school"] == False]

    if with_school.empty or without_school.empty:
        traffic_ratio = float("nan")
        check_ratio = float("nan")
    else:
        mean_traf_school = with_school["mean_traffic"].iloc[0]
        mean_traf_no_school = without_school["mean_traffic"].iloc[0]
        mean_check_school = with_school["mean_check"].iloc[0]
        mean_check_no_school = without_school["mean_check"].iloc[0]

        traffic_ratio = mean_traf_school / mean_traf_no_school if mean_traf_no_school != 0 else float("nan")
        check_ratio = mean_check_school / mean_check_no_school if mean_check_no_school != 0 else float("nan")

    records.append(
        {
            col_np: np_name,
            col_region: region_name,
            "size_group": size_group,
            "traffic_ratio": traffic_ratio,
            "check_ratio": check_ratio,
        }
    )

ratio_df = pd.DataFrame(records)
print("Размер ratio_df:", ratio_df.shape)
ratio_df.head()

Размер ratio_df: (3037, 5)


,Населенный пункт,Регион,size_group,traffic_ratio,check_ratio
0,"1-го отделения совхоза ""Масловский"" п",Воронежская обл,medium,NaN,NaN
1,1-я Моква д,Курская обл,medium,1.105263,0.49858
2,Абадзехская ст-ца,Адыгея Респ,small,NaN,NaN
3,Абаза г,Хакасия Респ,medium,NaN,NaN
4,Абакан г,Хакасия Респ,large_plus,1.066131,1.00619


## Шаг 5. Пивот: 6 чисел на НП

In [26]:
pivot = ratio_df.pivot_table(
    index=[col_np, col_region],
    columns="size_group",
    values=["traffic_ratio", "check_ratio"],
)
pivot.columns = [f"{metric}_{size}" for metric, size in pivot.columns]
pivot = pivot.reset_index().sort_values([col_region, col_np]).reset_index(drop=True)
print("Финальная таблица отношений:")
pivot.head()

Финальная таблица отношений:


,Населенный пункт,Регион,check_ratio_large_plus,check_ratio_medium,check_ratio_small,traffic_ratio_large_plus,traffic_ratio_medium,traffic_ratio_small
0,Майкоп г,Адыгея Респ,NaN,1.019576,1.051715,NaN,0.918438,0.698583
1,Тахтамукай аул,Адыгея Респ,NaN,1.080329,NaN,NaN,1.435785,NaN
2,Энем пгт,Адыгея Респ,NaN,0.799786,NaN,NaN,0.385974,NaN
3,Яблоновский пгт,Адыгея Респ,NaN,0.849199,NaN,NaN,0.235805,NaN
4,Барнаул г,Алтайский край,1.131623,0.988751,0.884846,1.062328,0.996061,1.230678


In [27]:
out_excel = Path("ri_gt70_school_ped_traffic_check_ratios_by_np_70_30.xlsx")
pivot.to_excel(out_excel, index=False)
print("Сохранён файл:", out_excel.resolve())

Сохранён файл: /home/kali_person/Загрузки/Проектный тур Дано/.venv/ri_gt70_school_ped_traffic_check_ratios_by_np_70_30.xlsx


## Шаг 6. One-sample t-test по таблице отношений (mean vs 1.0)

In [28]:
try:
    from scipy import stats
    HAVE_SCIPY = True
except ImportError:
    HAVE_SCIPY = False
    print("[WARN] SciPy не установлен — t-тесты выполняться не будут.")

if HAVE_SCIPY:
    metrics = [
        ("traffic_ratio_small",      "Трафик, маленькие магазины"),
        ("check_ratio_small",        "Средний чек, маленькие магазины"),
        ("traffic_ratio_medium",     "Трафик, средние магазины"),
        ("check_ratio_medium",       "Средний чек, средние магазины"),
        ("traffic_ratio_large_plus", "Трафик, большие+очень большие магазины"),
        ("check_ratio_large_plus",   "Средний чек, большие+очень большие магазины"),
    ]
    alpha = 0.05

    for col, desc in metrics:
        if col not in pivot.columns:
            print(f"[WARN] Нет колонки {col!r} ({desc}), пропуск.")
            continue
        values = pivot[col].dropna()
        n = len(values)
        if n < 3:
            print(f"[WARN] Слишком мало наблюдений для {desc}: n={n}")
            continue
        t_stat, p_val = stats.ttest_1samp(values, popmean=1.0)
        mean_val = values.mean()
        print(f"\n=== {desc} (Индекс_РИА_2024 70>x<30 ===")
        print(f"n = {n}")
        print(f"mean = {mean_val:.4f}")
        print(f"t-stat = {t_stat:.3f}")
        print(f"p-value = {p_val:.4g}")
        if p_val < alpha:
            print(f"→ Различие статзначимо при alpha={alpha}")
        else:
            print(f"→ Статзначимых отличий от 1.0 нет при alpha={alpha}")


=== Трафик, маленькие магазины (Индекс_РИА_2024 70>x<30 ===
n = 177
mean = 1.1819
t-stat = 4.444
p-value = 1.559e-05
→ Различие статзначимо при alpha=0.05

=== Средний чек, маленькие магазины (Индекс_РИА_2024 70>x<30 ===
n = 177
mean = 0.9633
t-stat = -2.775
p-value = 0.006111
→ Различие статзначимо при alpha=0.05

=== Трафик, средние магазины (Индекс_РИА_2024 70>x<30 ===
n = 402
mean = 1.2422
t-stat = 4.421
p-value = 1.266e-05
→ Различие статзначимо при alpha=0.05

=== Средний чек, средние магазины (Индекс_РИА_2024 70>x<30 ===
n = 402
mean = 0.9849
t-stat = -1.082
p-value = 0.28
→ Статзначимых отличий от 1.0 нет при alpha=0.05

=== Трафик, большие+очень большие магазины (Индекс_РИА_2024 70>x<30 ===
n = 161
mean = 1.0845
t-stat = 2.144
p-value = 0.03357
→ Различие статзначимо при alpha=0.05

=== Средний чек, большие+очень большие магазины (Индекс_РИА_2024 70>x<30 ===
n = 161
mean = 0.9936
t-stat = -0.416
p-value = 0.6778
→ Статзначимых отличий от 1.0 нет при alpha=0.05


## Шаг 7. Welch t-test по сырым данным (со школами vs без школ)

In [29]:
if HAVE_SCIPY:
    sizes = ["small", "medium", "large_plus"]
    alpha = 0.05
    for size in sizes:
        sub = base_df[base_df["size_group"] == size]
        if sub.empty:
            print(f"\n[WARN] Нет данных для size_group={size}")
            continue
        traf_with = sub.loc[sub["has_school"], col_traffic].dropna()
        traf_without = sub.loc[~sub["has_school"], col_traffic].dropna()
        check_with = sub.loc[sub["has_school"], col_check].dropna()
        check_without = sub.loc[~sub["has_school"], col_check].dropna()

        print(f"\n=== size_group = {size} (Индекс_РИА_2024 70>x<30) ===")
        print(f"n (traf with/without) = {len(traf_with)} / {len(traf_without)}")
        print(f"n (check with/without) = {len(check_with)} / {len(check_without)}")

        if len(traf_with) >= 3 and len(traf_without) >= 3:
            t_traf, p_traf = stats.ttest_ind(traf_with, traf_without, equal_var=False)
            print(f"Трафик пеший, в час: t = {t_traf:.3f}, p = {p_traf:.4g}")
            if p_traf < alpha:
                print(f"→ Различие по трафику статзначимо (alpha={alpha})")
            else:
                print(f"→ Статзначимых различий по трафику нет (alpha={alpha})")
        else:
            print("Недостаточно наблюдений для t-теста по трафику")

        if len(check_with) >= 3 and len(check_without) >= 3:
            t_check, p_check = stats.ttest_ind(check_with, check_without, equal_var=False)
            print(f"Средний чек: t = {t_check:.3f}, p = {p_check:.4g}")
            if p_check < alpha:
                print(f"→ Различие по среднему чеку статзначимо (alpha={alpha})")
            else:
                print(f"→ Статзначимых различий по среднему чеку нет (alpha={alpha})")
        else:
            print("Недостаточно наблюдений для t-теста по среднему чеку")
else:
    print("SciPy недоступен — Welch t-test не выполнялся")


=== size_group = small (Индекс_РИА_2024 70>x<30) ===
n (traf with/without) = 2440 / 5184
n (check with/without) = 2440 / 5184
Трафик пеший, в час: t = 8.564, p = 1.418e-17
→ Различие по трафику статзначимо (alpha=0.05)
Средний чек: t = -8.004, p = 1.464e-15
→ Различие по среднему чеку статзначимо (alpha=0.05)

=== size_group = medium (Индекс_РИА_2024 70>x<30) ===
n (traf with/without) = 6193 / 15973
n (check with/without) = 6193 / 15973
Трафик пеший, в час: t = 16.075, p = 1.536e-57
→ Различие по трафику статзначимо (alpha=0.05)
Средний чек: t = -18.893, p = 1.402e-78
→ Различие по среднему чеку статзначимо (alpha=0.05)

=== size_group = large_plus (Индекс_РИА_2024 70>x<30) ===
n (traf with/without) = 2342 / 4764
n (check with/without) = 2342 / 4764
Трафик пеший, в час: t = 1.419, p = 0.156
→ Статзначимых различий по трафику нет (alpha=0.05)
Средний чек: t = -5.174, p = 2.372e-07
→ Различие по среднему чеку статзначимо (alpha=0.05)
